# RAG
Il RAG consiste di due fasi principali:

1. Indexing: Carichiamo i dati grezzi e li splittiamo in chuncks più piccoli. Questi chunks vengono parsati e un modello di embedding crea i vettori che rappresentano il significato semantico dei chunks. Gli embeddings assieme ai chunks sono salvati in un vector database.

2. Retrieval: Inizia con una domanda dall'utente. Il modello crea un embedding per la domanda e cerca i chunks più simili all'embedding della domanda nel vector database. Questi chunks sono poi combinati con la query originale dell'utente per formare un prompt, il quale poi viene inviato all'LLM per generare una risposta all'utente. L'LLM processa tutta questa informazione e genera una risposta well-informed human-like all'utente.



Ora andremo a embeddizzare alcuni documenti riguardanti un ristorante fittizio chiamato Bella Vista, con informazione riguardo il proprietario e i piatti che offre e anche gli orari di apertura ecc. Tutta l'informazione che di solito si cerca in un website insomma.






In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embedding_function = OpenAIEmbeddings(model="text-embedding-3-large")

docs = [
    Document(
        page_content="Bella Vista is owned by Antonio Rossi, a renowned chef with over 20 years of experience in the culinary industry. He started Bella Vista to bring authentic Italian flavors to the community.",
        metadata={"source": "owner.txt"},
    ),
    Document(
        page_content="Bella Vista offers a range of dishes with prices that cater to various budgets. Appetizers start at $8, main courses range from $15 to $35, and desserts are priced between $6 and $12.",
        metadata={"source": "dishes.txt"},
    ),
    Document(
        page_content="Bella Vista is open from Monday to Sunday. Weekday hours are 11:00 AM to 10:00 PM, while weekend hours are extended from 11:00 AM to 11:00 PM.",
        metadata={"source": "restaurant_info.txt"},
    ),
    Document(
        page_content="Bella Vista offers a variety of menus including a lunch menu, dinner menu, and a special weekend brunch menu. The lunch menu features light Italian fare, the dinner menu offers a more extensive selection of traditional and contemporary dishes, and the brunch menu includes both classic breakfast items and Italian specialties.",
        metadata={"source": "restaurant_info.txt"},
    ),
]

# Create a vectorstore from the documents
db = Chroma.from_documents(docs, embedding_function)

# Dopo aver creato il vectorstore, lo convertiamo in un retriever
retriever = db.as_retriever(search_kwargs={"k": 4})


In [3]:
retriever.invoke("When are the opening hours?")

[Document(metadata={'source': 'restaurant_info.txt'}, page_content='Bella Vista is open from Monday to Sunday. Weekday hours are 11:00 AM to 10:00 PM, while weekend hours are extended from 11:00 AM to 11:00 PM.'),
 Document(metadata={'source': 'restaurant_info.txt'}, page_content='Bella Vista offers a variety of menus including a lunch menu, dinner menu, and a special weekend brunch menu. The lunch menu features light Italian fare, the dinner menu offers a more extensive selection of traditional and contemporary dishes, and the brunch menu includes both classic breakfast items and Italian specialties.'),
 Document(metadata={'source': 'owner.txt'}, page_content='Bella Vista is owned by Antonio Rossi, a renowned chef with over 20 years of experience in the culinary industry. He started Bella Vista to bring authentic Italian flavors to the community.'),
 Document(metadata={'source': 'dishes.txt'}, page_content='Bella Vista offers a range of dishes with prices that cater to various budgets

## RAG Prompt

In [4]:
from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)



In [5]:
# possiamo anche utilizzare un RAG prompt da langchain hub 

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = hub.pull("rlm/rag-prompt") # è esattamente lo stesso prompt di sopra

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

"""
from langchain_core.runnables import RunnablePassthrough

runnable = RunnablePassthrough()
print(runnable.invoke("Hello"))  # Output: "Hello"
"""

qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()   #  Passa il valore in input direttamente come output 
    }
    | prompt
    | ChatOpenAI(model="gpt-4o")
    | StrOutputParser()
)

qa_chain.invoke("When are the opening hours?")

c:\Users\felip\Desktop\LangGraph_Advanced\LangGraph-mio\.venv\lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'Bella Vista is open from 11:00 AM to 10:00 PM on weekdays and from 11:00 AM to 11:00 PM on weekends.'